## Basic usage: Create basic and timed subscription

This tutorial is about the Cython level of the package, which connects Python and C API. Though high-level Python classes provide the necessary functionality safely and conveniently, the user is not restricted to use low-level functions. 

### Import core functions
Here we deal with low level C styled api, so the import is slightly differ.

In [ ]:
from dxfeed.core import DXFeedPy as dxc
from dxfeed.core.utils.handler import DefaultHandler
from datetime import datetime  # for timed suscription
from dateutil.relativedelta import relativedelta

### Create connection
There are two ways at the moment to create connection: with token or with specifying connection address. Here we use the latter for simplicity. 

In [ ]:
con = dxc.dxf_create_connection('demo.dxfeed.com:7300')

### Create subscription
There are two types of subscriptions: ordinary for delivering stream data as-is and timed for conflated data. Except type of subscription you should provide type of events you want to get. Note: some event types, e.g. Candle, support only timed subscription.

In [ ]:
sub = dxc.dxf_create_subscription(con, 'Trade')
sub_timed = dxc.dxf_create_subscription_timed(con, 'Candle', int((datetime.now() - relativedelta(days=3)).timestamp()))

### Attach event handler
To process incoming data you have to define define `update(event)` method in your EventHandler child class. Or you may use DefaultHandler which stores upcoming data in deque of the length 100k. In this example we choose the latter.

In [ ]:
trade_handler = DefaultHandler()
candle_handler = DefaultHandler()

In [ ]:
sub.set_event_handler(trade_handler)
sub_timed.set_event_handler(candle_handler)

### Attach listener
A special function that processes incoming on the C level events should be initialized. There are default ones for each event type.

In [ ]:
dxc.dxf_attach_listener(sub)
dxc.dxf_attach_listener(sub_timed)

### Add tickers
Symbols that will be processed should be defined. For Candle event type along with base symbol, you should specify an aggregation period. You can also set price type. More details: https://kb.dxfeed.com/en/market-data-api/data-access-solutions/rest.html#candle-symbols.

In [ ]:
dxc.dxf_add_symbols(sub, ['AAPL', 'MSFT'])
dxc.dxf_add_symbols(sub_timed, ['AAPL{=d}', 'MSFT{=d}'])

### Access data
The DefaultHandler class has `get_list()` and `get_dataframe()` methods to access the data.

In [ ]:
trade_handler.get_list()[-5:]

In [ ]:
candle_handler.get_dataframe().head(3)

### Detach listener
When you are no longer interested in recieving data detach the listener

In [ ]:
dxc.dxf_detach_listener(sub)
dxc.dxf_detach_listener(sub_timed)

### Close connection

In [ ]:
dxc.dxf_close_connection(con)

### Transform data to pandas DataFrame

In [ ]:
df1 = trade_handler.get_dataframe()
df1.head(3)

In [ ]:
df2 = candle_handler.get_dataframe()
df2.head(3)